# **Deep Learning applied on Nairobi slum**

## Import libraries

In [169]:
import tensorflow as tf
import os, sys
import numpy as np
import h5py

In [170]:
from stackage import stackage

In [171]:
def import_hf5(data_path, data_name, get_name):
    hf = h5py.File(os.path.join(data_path, data_name), mode="r")
    data = hf.get(get_name)
    stack = np.array(data)
    hf.close()
    return stack

In [172]:
#code from https://github.com/tgrippa/Partimap/blob/cd72ecd2f3e32794eb3e0ee70399de1811b5399e/Notebooks/Data_preprocessing.ipynb
def normalise_01(image_data):
    image_data -= np.min(stack, axis=0)
    image_data /= (np.max(stack, axis=0) - np.min(stack, axis=0))
    return image_data

## Importing data

In [173]:
data_path = 'Images/S2/merge_RGBNIR_patch/'
data_name = "RGBNIR.hdf5"
path_raster = import_hf5(data_path,data_name,'raster')

patch_id_train = import_hf5(data_path,data_name,'id_train')
patch_id_test = import_hf5(data_path,data_name,'id_test')

patch_lulc_train = import_hf5(data_path,data_name,"lulc_train")
patch_lulc_test = import_hf5(data_path,data_name,"lulc_test")

In [174]:
patch_id_train.shape

(3643, 1)

## Separate train patchs to test patchs

In [175]:
train_path = []
test_path = []

patch_id_train = patch_id_train.reshape(1,patch_id_train.size)
patch_id_train_list = patch_id_train.tolist()[0]
patch_id_test = patch_id_test.reshape(1,patch_id_test.size)
patch_id_test_list = patch_id_test.tolist()[0]

for path in path_raster : #récupération des id des images
    image = path.split(b'/')[-1]
    image_id = int(image.split(b'_')[-2])
    if image_id in patch_id_train_list :
        train_path.append(path)
    elif image_id in patch_id_test_list :
        test_path.append(path)

In [195]:
stack = stackage(train_path)
stack_train = normalise_01(stack)
stack = stackage(test_path)
stack_test = normalise_01(stack)
print(stack_test[0:,:,:,])

[[[[1136.  749.  490. 1743.]
   [1408. 1034.  720. 2520.]
   [1272. 1110.  801. 2480.]
   ...
   [1488. 1418. 1272. 1992.]
   [1090. 1042.  817. 2546.]
   [ 710.  692.  433. 2802.]]

  [[1134.  973.  666. 2604.]
   [1196. 1064.  772. 2878.]
   [ 985.  940.  661. 2778.]
   ...
   [1434. 1318. 1150. 2400.]
   [1034.  875.  599. 2394.]
   [ 869.  858.  576. 2540.]]

  [[ 641.  672.  405. 2731.]
   [ 561.  601.  351. 2698.]
   [ 597.  608.  333. 2677.]
   ...
   [1304. 1011.  810. 2084.]
   [1152.  978.  684. 2472.]
   [1332. 1074.  725. 2574.]]

  ...

  [[ 970.  805.  501. 2364.]
   [ 953.  788.  502. 2376.]
   [ 878.  760.  471. 2314.]
   ...
   [1100.  877.  668. 1772.]
   [ 761.  699.  465. 1266.]
   [ 971.  833.  720. 1110.]]

  [[1052.  886.  539. 2359.]
   [ 999.  824.  490. 2385.]
   [ 830.  767.  485. 2216.]
   ...
   [ 987.  785.  555. 1665.]
   [ 733.  686.  523. 1183.]
   [ 834.  684.  520. 1038.]]

  [[1358. 1062.  694. 2311.]
   [1216.  958.  623. 2310.]
   [ 862.  785.  496

In [191]:
print(len(patch_id_train_list))
print(patch_id_test.shape)
print(len(test_path))
print(path_raster)

3643
(1, 911)
911
[b'Images/S2/merge_RGBNIR_patch/merge_RGBNIR_52739_0.tif'
 b'Images/S2/merge_RGBNIR_patch/merge_RGBNIR_52740_0.tif'
 b'Images/S2/merge_RGBNIR_patch/merge_RGBNIR_52741_0.tif' ...
 b'Images/S2/merge_RGBNIR_patch/merge_RGBNIR_70006_1.tif'
 b'Images/S2/merge_RGBNIR_patch/merge_RGBNIR_70007_0.tif'
 b'Images/S2/merge_RGBNIR_patch/merge_RGBNIR_70008_0.tif']


In [197]:
print(patch_lulc_train.shape)
print(stack_train.shape)
print(patch_lulc_train)

(3643, 1)
(3643, 10, 10, 4)
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


## Starting deep learning

In [201]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(10, 10, 4)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(stack_train, patch_lulc_train, epochs=5)
test_loss, test_acc = model.evaluate(stack_test, patch_lulc_test)
print(test_acc)

Epoch 1/5


UnimplementedError: Graph execution error:

Detected at node 'sequential_12/conv2d_24/Conv2D' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 972, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 505, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyncio/base_